In [356]:
import pandas as pd
train_dataset = pd.read_csv('training-set.csv/training-set.csv',delimiter=',')
judge_dataset = pd.read_csv('judge-set.csv/judge-set.csv',delimiter=',')


In [357]:
train_dataset.head()

,Tick#,offsetTime,IMU_ATTI(0):numSats,IMU_ATTI(0):barometer:Raw,IMU_ATTI(0):barometer:Smooth,IMU_ATTI(0):accel:X,IMU_ATTI(0):accel:Y,IMU_ATTI(0):accel:Z,IMU_ATTI(0):accel:Composite,IMU_ATTI(0):gyro:X,...,MotorCtrl:PWM:RFront,MotorCtrl:PWM:LFront,MotorCtrl:PWM:LBack,MotorCtrl:PWM:RBack,AirCraftCondition:fsmState,AirCraftCondition:nearGround,AirCraftCondition:landState,AirCraftCondition:UP_acc_t,AirCraftCondition:UP_TF_t,Operating_Phase
0,339062105,0.000,14,1188.3093,1188.0585,-0.007624,-0.162505,-0.980750,0.994151,-0.013848,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
1,339314363,0.066,14,1188.1595,1188.0468,-0.007572,-0.161164,-0.981163,0.994339,0.049079,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
2,339465911,0.105,14,1188.1595,1188.0468,-0.007572,-0.161164,-0.981163,0.994339,0.049079,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
3,339648466,0.152,14,1188.2122,1188.0359,-0.008646,-0.153160,-0.982913,0.994812,0.535372,...,NaN,NaN,NaN,NaN,1,True,1,0.0,0,Resting
4,339821470,0.197,14,1188.2122,1188.0359,-0.008646,-0.153160,-0.982913,0.994812,0.535372,...,NaN,NaN,NaN,NaN,1,True,1,0.0,0,Resting


In [358]:
total_number_of_rows = train_dataset.shape[0] 
total_number_of_columns = train_dataset.shape[1]
number_of_columns_non_numeric = len(train_dataset.select_dtypes(exclude=['number']).columns)
missing_values = train_dataset.isnull().sum()
missing_columns = [(column, missing_number) for column, missing_number in missing_values.items() if missing_number>0]
missing_columns.sort(key=lambda x: x[1], reverse=True)
summarizing =   f"total number of rows = {total_number_of_rows}\n"\
                f"total number of columns = {total_number_of_columns}\n"\
                f"number of columns having non-numeric values = {number_of_columns_non_numeric}\n"\
                f"columns with missing values = {missing_columns}\n"
print(summarizing)

total number of rows = 5085
total number of columns = 142
number of columns having non-numeric values = 5
columns with missing values = [('General:absoluteHeight', 5085), ('OA:frontDistance', 224), ('IMU_ATTI(1):directionOfTravel[true]', 32), ('IMU_ATTI(0):directionOfTravel[mag]', 8), ('IMU_ATTI(0):directionOfTravel[true]', 8), ('MotorCtrl:PWM:RFront', 5), ('MotorCtrl:PWM:LFront', 5), ('MotorCtrl:PWM:LBack', 5), ('MotorCtrl:PWM:RBack', 5), ('General:relativeHeight', 1)]



In [359]:
column_name = 'Operating_Phase'
X = train_dataset.drop(columns=[column_name])
y = train_dataset[column_name]

In [360]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12346)
# Define the list of columns to extract
columns_to_drop = [
    'General:absoluteHeight',
    'GPS(0):Date',
    'GPS(0):Time',
    'GPS:dateTimeStamp',
    'IMUEX(0):err',
    'IMUEX(1):err',
    'IMUEX(0):rtk_Alti',
    'IMUEX(1):rtk_Alti'
]

X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)


In [361]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
categorical_columns = X_train.select_dtypes(exclude=['number']).columns
X_train_ohe = pd.get_dummies(X_train,  columns=categorical_columns, dtype=int)
X_test_ohe = pd.get_dummies(X_test,  columns=categorical_columns, dtype=int)
print(categorical_columns)

Index(['AirCraftCondition:nearGround'], dtype='object')


In [362]:
missing_values = X_train_ohe.isnull().sum()
missing_counts = pd.DataFrame({'variable_name': missing_values.index, 'num_of_missing_values': missing_values.values})
X_train_ohe_imputed = X_train_ohe.fillna(X_train_ohe.mean())
X_test_ohe_imputed = X_test_ohe.fillna(X_test_ohe.mean())

In [363]:

missing_values = X_train_ohe_imputed.isnull().sum()
missing_columns = [(column, missing_number) for column, missing_number in missing_values.items() if missing_number>0]
print(missing_columns, X_train_ohe_imputed.shape[1])

[] 134


In [364]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)
correlation_matrix = X_train_ohe_imputed.corrwith(pd.Series(y_train_encoded))

correlation_matrix.dropna(inplace=True)

top75_corr = correlation_matrix.abs()#.nlargest(70)
top75_df = pd.DataFrame({'variable': top75_corr.index, 'corr_score': top75_corr.values})
print(top75_df)


                               variable  corr_score
0                                 Tick#    0.004182
1                            offsetTime    0.004182
2                   IMU_ATTI(0):numSats    0.015660
3             IMU_ATTI(0):barometer:Raw    0.022762
4          IMU_ATTI(0):barometer:Smooth    0.020224
..                                  ...         ...
116          AirCraftCondition:fsmState    0.009518
117         AirCraftCondition:landState    0.000189
118          AirCraftCondition:UP_acc_t    0.008056
119  AirCraftCondition:nearGround_False    0.017976
120   AirCraftCondition:nearGround_True    0.017976

[121 rows x 2 columns]


c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [365]:
y_train_encoded

array([0, 0, 0, ..., 1, 0, 0])

In [351]:
top75_variables = top75_df['variable'].tolist()

X_train_t75 = X_train_ohe_imputed[top75_variables]
X_test_t75 = X_test_ohe_imputed[top75_variables]


In [352]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled_mm = scaler.fit_transform(X_train_t75)
X_test_scaled_mm = scaler.fit_transform(X_test_t75)

In [353]:
from sklearn.preprocessing import StandardScaler
scaler_s = StandardScaler()
X_train_scaled_std = scaler_s.fit_transform(X_train_t75)
X_test_scaled_std = scaler_s.fit_transform(X_test_t75)
print(X_train_scaled_std,X_test_scaled_std)

[[ 0.72209135 -0.71522247 -0.86133174 ...  1.15873809  0.05969431
   0.07786767]
 [ 1.09306424 -0.10311352  0.04210052 ...  0.72069587  0.05969431
   4.99017137]
 [ 1.09306424  0.31642949  0.02175815 ...  0.72069587  0.05969431
   4.9793457 ]
 ...
 [-0.45707249  0.16495654  0.11283046 ...  0.98807524  1.54737017
   0.00921608]
 [-0.49681958  0.84004944 -0.41783064 ... -0.2867058   0.05969431
   0.22862339]
 [ 1.06656618 -0.20089783  0.20626255 ...  0.67274561  0.05969431
  -0.12321216]] [[-0.84350096 -0.28763791 -0.58829775 ...  1.18600539 -0.01077055
  -0.02467509]
 [-0.79039334 -0.30643895 -0.05639939 ... -0.90990373 -1.5330086
  -0.14740933]
 [ 0.68334308  0.72202692  0.49110607 ...  1.18600539 -0.01077055
  -0.07255667]
 ...
 [-0.0999943   0.91138171  0.47090749 ... -0.30667353 -0.01077055
   0.12067836]
 [ 0.55057403  0.24902365 -0.11077019 ... -0.02587189 -0.01077055
  -0.06448214]
 [-1.812715    0.05314336 -0.1504278  ...  0.68413137 -0.01077055
   0.09531411]]


In [354]:
"""from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Create an SVM classifier
svm_classifier = SVC()

# Fit the SVM classifier on the training data
svm_classifier.fit(X_train_t75, y_train_encoded)

# Predict on the test data
y_pred = svm_classifier.predict(X_test_t75)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier()

# Fit the Random Forest classifier on the training data
rf_classifier.fit(X_train_t75, y_train_encoded)

# Predict on the test data
y_pred_rf = rf_classifier.predict(X_test_t75)

# Calculate and print the accuracy
accuracy_rf = accuracy_score(y_test_encoded, y_pred_rf)
print(f"Random Forest Test Accuracy: {accuracy_rf * 100:.2f}%")
"""

'from sklearn.svm import SVC\nfrom sklearn.metrics import accuracy_score\n\n# Create an SVM classifier\nsvm_classifier = SVC()\n\n# Fit the SVM classifier on the training data\nsvm_classifier.fit(X_train_t75, y_train_encoded)\n\n# Predict on the test data\ny_pred = svm_classifier.predict(X_test_t75)\n\n# Calculate and print the accuracy\naccuracy = accuracy_score(y_test_encoded, y_pred)\nprint(f"Test Accuracy: {accuracy * 100:.2f}%")\n\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import accuracy_score\n\n# Create a Random Forest classifier\nrf_classifier = RandomForestClassifier()\n\n# Fit the Random Forest classifier on the training data\nrf_classifier.fit(X_train_t75, y_train_encoded)\n\n# Predict on the test data\ny_pred_rf = rf_classifier.predict(X_test_t75)\n\n# Calculate and print the accuracy\naccuracy_rf = accuracy_score(y_test_encoded, y_pred_rf)\nprint(f"Random Forest Test Accuracy: {accuracy_rf * 100:.2f}%")\n'

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

num_classes = 5
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_t75.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(units=50, return_sequences=True))

model.add(Flatten())
model.add(Dense(units=num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

num_classes = 5 
y_train_encoded = to_categorical(y_train_encoded, num_classes=num_classes)

model.fit(X_train_t75, y_train_encoded, epochs=10, batch_size=32)

y_test_encoded = to_categorical(y_test_encoded, num_classes=num_classes)
test_loss, test_accuracy = model.evaluate(X_test_t75, y_test_encoded)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


ModuleNotFoundError: No module named 'tensorflow.python'

In [315]:
judge_data = judge_dataset.drop(columns=columns_to_drop)

In [294]:
categorical_columns = judge_data.select_dtypes(exclude=['number']).columns
judge_data_ohe = pd.get_dummies(judge_data,  columns=categorical_columns, dtype=int)
missing_values = judge_data_ohe.isnull().sum()
missing_counts = pd.DataFrame({'variable_name': missing_values.index, 'num_of_missing_values': missing_values.values})
judge_data_ohe_imputed = X_train_ohe.fillna(X_train_ohe.mean())
print(judge_data_ohe.columns)
judge_data_ohe['AirCraftCondition:nearGround_False'] = 0
print(judge_data_ohe.head())

Index(['Tick#', 'offsetTime', 'IMU_ATTI(0):numSats',
       'IMU_ATTI(0):barometer:Raw', 'IMU_ATTI(0):barometer:Smooth',
       'IMU_ATTI(0):accel:X', 'IMU_ATTI(0):accel:Y', 'IMU_ATTI(0):accel:Z',
       'IMU_ATTI(0):accel:Composite', 'IMU_ATTI(0):gyro:X',
       ...
       'IMU_ATTI(1):temperature', 'MotorCtrl:PWM:RFront',
       'MotorCtrl:PWM:LFront', 'MotorCtrl:PWM:LBack', 'MotorCtrl:PWM:RBack',
       'AirCraftCondition:fsmState', 'AirCraftCondition:landState',
       'AirCraftCondition:UP_acc_t', 'AirCraftCondition:UP_TF_t',
       'AirCraftCondition:nearGround_True'],
      dtype='object', length=133)
       Tick#  offsetTime  IMU_ATTI(0):numSats  IMU_ATTI(0):barometer:Raw  \
0  143222258       0.000                   11                   1230.502   
1  143393138       0.044                   11                   1230.502   
2  143566452       0.089                   11                   1230.404   
3  143758752       0.139                   11                   1230.404   
4  1

In [316]:
judge_data_t75 = judge_data_ohe[top75_variables]
X_judge_scaled_std = scaler_s.fit_transform(judge_data_t75)
 

In [317]:
import numpy as np
predicted_probabilities = model.predict(X_judge_scaled_std) 
class_labels = ['Resting', 'Taking-Off', 'Hovering', 'Climbing', 'Landing']
predicted_operating_phase_labels = [class_labels[np.argmax(prediction)] for prediction in predicted_probabilities]
new_dataset = judge_data[["Tick#"]].copy()
new_dataset["Operating_Phase"] = predicted_operating_phase_labels
new_dataset.to_csv("new_dataset_with_tick_and_operating_phase.csv", index=False)

207/207 [==============================] - 3s 13ms/step
